In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import numpy as np
import pandas as pd
import sklearn
import collections
from functools import partial
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.impute import SimpleImputer
# from hyperopt import fmin, tpe, hp, rand, Trials, space_eval

In [2]:
def concate(array, log_array):
    dataset = []
    for i in range(len(array)):
        t1 = array[i][0]
        for j in range(len(log_array)):
            if log_array[j][0] == t1:
                concat = np.concatenate((array[i], log_array[j]))
                dataset.append(concat)
            else:
                pass
    return np.array(dataset)

def impute_values(data):
    for c in data.columns:
        data[c].replace([np.inf, -np.inf], np.nan, inplace = True)
#         data[c].replace([np.inf, -np.inf, np.nan], data[c].mean(), inplace = True)
    for c in data.columns:
        data[c].fillna(value = data[c].mean(), inplace = True)
    return data

def rf_objective(parameter, feature, label):
    loss = -cross_val_score(RandomForestRegressor(**parameter, n_jobs = -1),
                            feature, label, cv = 5, scoring = 'neg_mean_absolute_error').mean()
    return loss

# def impute_value(dataframe):
#     for c in dataframe.columns:
#         dataframe[c].replace([np.inf, -np.inf], value = dataframe[c].mean(), inplace = True)
#     return dataframe

In [ ]:
rf_parameter = {"n_estimators": hp.choice('n_estimators', np.linspace(10, 200, num = 20, dtype = np.int32)),
            "max_depth": hp.choice('max_depth', np.linspace(1, 20, num = 20)),
            'min_samples_leaf': hp.choice('min_samples_leaf', np.linspace(2, 30, num = 25, dtype = np.int32))}

In [3]:
data_path = '/content/drive/My Drive/project/smart_manufacturing/csv_file'
train = pd.read_csv(os.path.join(data_path, 'encode_train_features.csv'))
test = pd.read_csv(os.path.join(data_path, 'encode_test_features.csv'))
train_log = pd.read_csv(os.path.join(data_path, 'train_log_feature.csv'))
test_log = pd.read_csv(os.path.join(data_path, 'test_log_feature_1.csv'))
train.drop(['Unnamed: 0', 'P_ID'], axis = 1, inplace = True)
test.drop(['Unnamed: 0'], axis = 1, inplace = True)
train_log.drop(['Unnamed: 0'], axis = 1, inplace = True)
train = impute_values(train)
train_log = impute_values(train_log)
test = impute_values(test)
test_log = impute_values(test_log)
# test_log.drop(['Unnamed: 0'], axis = 1, inplace = True)
target = np.load('/content/drive/My Drive/project/smart_manufacturing/npy_file/target_array.npy')

In [4]:
train_features = pd.concat([train, train_log], axis = 1)
# train_features = SimpleImputer().fit_transform(train_features)
test = np.array(test)
test_log = np.array(test_log)
test_features = concate(test, test_log)
test_features = np.delete(test_features, [0], axis = 1)
# test_features = SimpleImputer().fit_transform(test_features)
# test_features = impute_value(test_features)
# train_features = MinMaxScaler().fit_transform(train_features)
# test_features = MinMaxScaler().fit_transform(test_features)

In [5]:
test_features.shape

(5723, 1514)

In [6]:
predict_data = np.zeros((5723, 1))
for i in range(4):
#     objective = partial(rf_objective, feature = np.array(train_features), label = target[:,i])
#     min_objective = fmin(fn = objective, space = rf_parameter, algo = tpe.suggest, max_evals = 15, trials = Trials())
#     values = space_eval(rf_parameter ,min_objective).values()
#     best_parameter = np.int32(list(values))
#     print('Target is : ', target)
#     print("best rf estimate parameters" , space_eval(rf_parameter, min_objective))
#     print("=============================")
    best_rf = RandomForestRegressor(n_estimators = 125, max_depth = 15, 
                                         min_samples_leaf =  5, 
                                         n_jobs = -1)
#     # 訓練
    best_rf.fit(np.array(train_features), target[:,i])
    # 預測
    predict = best_rf.predict(test_features)
    predict = np.reshape(predict, (5723, 1))
    predict_data = np.concatenate((predict_data, predict), axis = 1)

predict_data = predict_data[:,1:]
predict_dataframe = pd.DataFrame(predict_data, columns = targets)

ValueError: ignored

In [ ]:
predict_dataframe

In [ ]:
predict_dataframe

In [ ]:
submmit = pd.concat([test['id'], predict_dataframe], axis = 1)
submmit

In [ ]:
submmit.to_csv('/home/motionlab/Desktop/weitai/project/test result/second_trial.csv', index = False)